In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"srishtidixit0811","key":"1a3e778ca6d47e5f829497a31cb2aea1"}'}

In [2]:
!pip install kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d thienkhonghoc/affectnet

Dataset URL: https://www.kaggle.com/datasets/thienkhonghoc/affectnet
License(s): unknown
 84% 1.47G/1.75G [01:07<00:12, 23.6MB/s]

In [ ]:
!unzip -q /content/affectnet.zip -d /content/affectnet > /dev/null 2>&1

In [ ]:
!pip install mediapipe opencv-python numpy pandas tensorflow keras albumentations

In [ ]:
!pip install --upgrade mediapipe opencv-python tqdm


In [ ]:
import os
os._exit(00)

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tqdm import tqdm  # Progress bar

# Enable GPU processing
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is enabled and available.")
    except RuntimeError as e:
        print("GPU setup error:", e)
else:
    print("No GPU found, running on CPU.")

# Set Correct Paths
DATASET_PATH = "/content/affectnet/AffectNet/"  # Main dataset directory
PROCESSED_PATH = "/content/AffectNet_Preprocessed/"  # Where processed images will be saved
BATCH_SIZE = 5000  # Reduce batch size if Colab runs out of memory

# Create directory for processed images
os.makedirs(PROCESSED_PATH, exist_ok=True)

# Initialize Mediapipe Face Detector
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)

# Function to detect and process faces in batches using GPU
@tf.function  # JIT compilation for GPU acceleration
def process_batch(batch_files, label, subset, save_path):
    for img_file in tqdm(batch_files, desc=f"Processing {subset}/{label}"):
        img_path = os.path.join(DATASET_PATH, subset, label, img_file)
        save_subset_path = os.path.join(save_path, subset, label)
        os.makedirs(save_subset_path, exist_ok=True)  # Ensure subset directories exist

        save_file_path = os.path.join(save_subset_path, img_file)

        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if image is not valid

        # Convert to RGB and process with Mediapipe
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_detection.process(img_rgb)

        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                h, w, _ = img.shape
                x, y, w, h = int(bboxC.xmin * w), int(bboxC.ymin * h), int(bboxC.width * w), int(bboxC.height * h)

                # Ensure face bounding box is valid
                if x < 0 or y < 0 or w <= 10 or h <= 10:
                    continue  # Skip images with invalid face detection

                face = img[y:y + h, x:x + w]

                # Ensure the cropped face is not empty
                if face is None or face.shape[0] == 0 or face.shape[1] == 0:
                    continue  # Skip empty images

                # Use OpenCV CUDA for GPU-accelerated resizing
                if cv2.cuda.getCudaEnabledDeviceCount() > 0:
                    gpu_img = cv2.cuda_GpuMat()
                    gpu_img.upload(face)
                    gpu_img = cv2.cuda.resize(gpu_img, (224, 224))
                    face = gpu_img.download()
                else:
                    face = cv2.resize(face, (224, 224))

                # Convert to grayscale
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

                # Save processed image
                cv2.imwrite(save_file_path, face)

# Process images in batches for train, val, and test sets
for subset in ["train", "val", "test"]:
    subset_path = os.path.join(DATASET_PATH, subset)

    if not os.path.exists(subset_path):
        print(f"Warning: {subset_path} does not exist. Skipping...")
        continue  # Skip if directory does not exist

    for label in os.listdir(subset_path):
        label_path = os.path.join(subset_path, label)
        save_label_path = os.path.join(PROCESSED_PATH, subset, label)

        if not os.path.isdir(label_path):
            continue  # Skip files, only process directories

        image_files = os.listdir(label_path)

        # Process in chunks of BATCH_SIZE
        for i in range(0, len(image_files), BATCH_SIZE):
            batch_files = image_files[i:i + BATCH_SIZE]
            process_batch(batch_files, label, subset, PROCESSED_PATH)
            print(f"Processed {len(batch_files)} images in batch for {subset}/{label}")


In [ ]:
!pip install tensorflow keras


In [ ]:
import os
import matplotlib.pyplot as plt

PROCESSED_PATH = "/content/AffectNet_Preprocessed/train/"

class_counts = {label: len(os.listdir(os.path.join(PROCESSED_PATH, label))) for label in os.listdir(PROCESSED_PATH)}

# Plot class distribution
plt.bar(class_counts.keys(), class_counts.values())
plt.xlabel("Emotion Class")
plt.ylabel("Number of Images")
plt.title("Class Distribution in AffectNet")
plt.xticks(rotation=45)
plt.show()

print("Class Distribution:", class_counts)


In [ ]:
#v4.1
#Increasing epochs from 20


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Check GPU Status
print("Checking GPU status...")
!nvidia-smi

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("GPU is enabled.")
else:
    print("No GPU found, running on CPU.")

# Enable mixed precision for speedup
tf.keras.mixed_precision.set_global_policy("mixed_float16")

PROCESSED_PATH = "/content/AffectNet_Preprocessed/"
checkpoint_path = "/content/best_augmented_model_4.2.keras"  # Load the saved model

# Data Augmentation (Enhance Variability)
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.5, 1.5],
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Load Dataset
train_generator = train_datagen.flow_from_directory(
    PROCESSED_PATH + "train",
    target_size=(224, 224),
    batch_size=16,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    PROCESSED_PATH + "val",
    target_size=(224, 224),
    batch_size=16,
    class_mode="categorical"
)

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print("Classes:", class_labels)

# Load Saved Model from Epoch 30
if os.path.exists(checkpoint_path):
    print("Loading previous checkpoint...")
    model = load_model(checkpoint_path)
    epochs_completed = 30  # Resume from epoch 31
else:
    print("No checkpoint found. Exiting...")
    exit()

# Define Callbacks
early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True, monitor="val_accuracy")
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6, verbose=1)

# Resume Training in Batches of 5 Epochs
epochs_completed = 30  # Start from 30th epoch
max_epochs = 50  # Train until 50 epochs max

while epochs_completed < max_epochs:
    print(f"Resuming training from epoch {epochs_completed + 1} to {epochs_completed + 5}")

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs_completed + 5,
        initial_epoch=epochs_completed,
        callbacks=[early_stop, checkpoint, reduce_lr]
    )

    # Save Model Progress After Every 5 Epochs
    model.save(checkpoint_path)

    epochs_completed += 5

print("Training completed.")


Checking GPU status...
Wed Mar  5 19:22:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P0             29W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+------------------------

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 31/35
2252/2252 ━━━━━━━━━━━━━━━━━━━━ 828s 295ms/step - accuracy: 0.5179 - loss: 1.2851 - val_accuracy: 0.5514 - val_loss: 1.2520 - learning_rate: 2.5000e-06
Epoch 32/35
2252/2252 ━━━━━━━━━━━━━━━━━━━━ 544s 241ms/step - accuracy: 0.5233 - loss: 1.2762 - val_accuracy: 0.5476 - val_loss: 1.2402 - learning_rate: 2.5000e-06
Epoch 33/35
2252/2252 ━━━━━━━━━━━━━━━━━━━━ 540s 240ms/step - accuracy: 0.5242 - loss: 1.2721 - val_accuracy: 0.5463 - val_loss: 1.2487 - learning_rate: 2.5000e-06
Epoch 34/35
2252/2252 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.5259 - loss: 1.2676
Epoch 34: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.
2252/2252 ━━━━━━━━━━━━━━━━━━━━ 540s 240ms/step - accuracy: 0.5259 - loss: 1.2676 - val_accuracy: 0.5476 - val_loss: 1.2638 - learning_rate: 2.5000e-06
Epoch 35/35
2252/2252 ━━━━━━━━━━━━━━━━━━━━ 536s 238ms/step - accuracy: 0.5276 - loss: 1.2678 - val_accuracy: 0.5424 - val_loss: 1.2413 - learning_rate: 1.2500e-06
Resuming training from epoch 